https://www.fukushihoken.metro.tokyo.lg.jp/hodo/saishin/hassei.html


In [67]:
import os
from WebScrapingTool import Base_UserFunction as uf
import json
import urllib

In [68]:
# https://www.shibutan-bloomers.com/python_library_pdfminer-six/2124/#21PDFJupyterNotebook

from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LAParams
from io import StringIO
import jaconv
from datetime import date
from japanera import Japanera, EraDate
import unicodedata

janera = Japanera()

def parse(filePath, type):
    getText = ''
    retData = '{'
    try:
        fp = open(filePath, 'rb')
        outfp = StringIO()

        rmgr = PDFResourceManager()
        lprms = LAParams()
        device = TextConverter(rmgr, outfp, laparams=lprms)
        iprtr = PDFPageInterpreter(rmgr, device)

        for page in PDFPage.get_pages(fp):
            iprtr.process_page(page)
        getText = outfp.getvalue()
    finally:
        device.close()
        outfp.close()
        fp.close()

    baseList = getText.split()

    # date
    _date = jaconv.z2h(baseList[1], kana=False, ascii=False, digit=True)
    _date = _date.replace('◆', '')
    _date = _date[:_date.find('日') + 1]
    _date = janera.strptime(_date, "%-E%-kO年%-km月%-kd日")
    _date = _date[0].strftime('%Y/%m/%d')
    print(_date)
    retData += '"date" : "' + _date

    idx = -1
    mode = (
        '患者の発生状況', 
        '行政検査数', 
        '都の独自検査件数', 
        '年代', 
        '都内発生数', 
        '区市町村別患者数', 
        '都内コロナウイルス陽性患者'
    )

    
    for l in baseList[2:]:
        if mode[idx + 1] in l:
            idx += 1
            if idx + 1 >= len(mode):
                break
            retData += '", "' + mode[idx] + '" : "'
        
        tmp = l.replace('(', '').replace(')', '').replace(',', '').replace('%', '')
        if tmp.encode('utf-8').isdigit():
            retData += tmp + ","
        elif tmp.isascii() and '.' in tmp:
            retData += tmp + ","

    retData += '"}'
    print(retData)
    return retData


In [69]:
def main():
    print("\n[Start]"  + uf.getNowTime() + '\n')

    #設定ファイルから必要な情報を取得する
    #タグ
    tag_debug = '[a]'
    tag_saveFolder = '[b]'
    tag_loadFileName = '[c]'
    tag_saveFileName = '[j]'
    
    isDebug = False
    _saveFolder = ''
    _loadFileName = ''
    _saveFileName = ''
    
    try:
        with open('_Setting.txt', mode='r') as f:
            lines = f.readlines()
            for l in lines:
                if l.startswith(tag_debug, 0, 3):
                    if (l.replace(tag_debug, '').rstrip()).lower() == 'true':
                        isDebug = True
                    else:
                        isDebug = False                    

                if l.startswith(tag_saveFolder, 0, 3):
                    _saveFolder = l.replace(tag_saveFolder, '').rstrip()
             
                if l.startswith(tag_loadFileName, 0, 3):
                    _loadFileName = l.replace(tag_loadFileName, '').rstrip()
             
                if l.startswith(tag_saveFileName, 0, 3):
                    _saveFileName = l.replace(tag_saveFileName, '').rstrip()
             
    except:
        print('[!!!ERROR!!!] Read Setting.text')
        return        
    
    if len(_saveFolder) <= 0:
        print('[!!!ERROR!!!] Image data storage folder is None!')
        return  
    
    baseFile =_saveFolder + "/" + _loadFileName
    print(baseFile)

    saveFile =_saveFolder + "/" + _saveFileName
    print(saveFile)
    
    # ファイルを開く
    updateList = list()
    parseList = list()
    with open(baseFile, mode='r') as f:
        cnt = 0
        for line in f:
            l = line
            j = json.loads(line)
            type = j['type']
            fileName = j['name']
            isParse = j['isParse']
            if isParse == "False":
                parseList.append(parse(_saveFolder + "/" + fileName, type))
                cnt += 1
                print("...Parse PDF : " + fileName + '  ' + str(cnt))
                #l = l.replace('"isParse" : "False"', '"isParse" : "True"')
            updateList.append(l)

        print('\n...Get Size :' + str(cnt) + '\n')

    # ファイル更新
    with open(baseFile, mode='w') as f:
        for line in updateList:
                uf.fileWrite(f, line)

    # パースしたデータを追加
    with open(saveFile, mode='a') as f:
        for line in parseList:
                uf.fileWrite(f, line)

    print("\n[ End ]"  + uf.getNowTime() + '\n')
    
    
if __name__ == '__main__':
    main()


[Start]20210724011837

#data/dataList.json
#data/parseList.csv
2021/07/20
1180.0
790.6
149.3
6700.7
{"date" : "2021/07/20", "患者の発生状況" : "1387,768,619,0,36,9,1180.0,790.6,149.3,", "行政検査数" : "6700.7,6465,2503,11134,", "都の独自検査件数" : "667,541,23,29001,5857,3163,5475,43496,6214,", "年代" : "75,78,448,300,234,179,45,20,6,2,0,0,", "都内発生数" : "191230,60,", "区市町村別患者数" : "996,926,3086,2924,6143,5843,10144,9902,2885,2662,3471,3298,3906,3635,6646,6115,5613,5271,5237,4947,9296,8706,13890,12974,5393,5133,6260,5902,7699,7170,5077,4759,4345,4092,3068,2899,7180,6744,8363,7855,8706,8203,6127,5886,8133,7688,4438,4288,1540,1452,1505,1397,1797,1677,870,837,2115,2037,883,843,2217,2087,3068,2886,1099,1028,1301,1219,1276,1240,948,899,958,880,511,487,533,507,699,648,516,481,457,427,769,722,417,385,1033,994,609,580,363,354,606,582,1855,1741,192,187,116,108,10,10,25,25,53,48,0,0,2,2,1,1,5,4,2,2,9,9,0,0,4,4,15297,15121,80,79,"}
...Parse PDF : 20210721_0721-30-01.pdf  1
2021/07/21
{"date" : "2021/07/21", "患者の発生状況" : 